In [2]:
# Data Imports
import numpy as np
import pandas as pd


# Math
import math

# Plot imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Machine Learning Imports
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# For evaluating our ML results
from sklearn import metrics

# Dataset Import
import statsmodels.api as sm

In [3]:
# Reading csv files
# women = pd.read_csv('athletes-women.csv')
# men = pd.read_csv('athletes-men.csv')
# all_ath = pd.read_csv('athletes-all.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [4]:
#ensure IDs are int
# women['athlete_id']=women['athlete_id'].astype(int)
# men['athlete_id']=men['athlete_id'].astype(int)
# all_ath['athlete_id']=women['athlete_id'].astype(int)
# #drop nas
# all_ath = all_ath[all_ath['name'].notna()]
# #drop columns 
# all_ath.drop(columns=['team', 'region','affiliate','gender','helen','grace','filthy50','fgonebad','eat','background'])
# #Left join to get all rows in all_ath that are not in women
# df_all = all_ath.merge(women.drop_duplicates(), on=['athlete_id'], how='left', indicator=True)
# df_all.head()

In [34]:
# Reading csv files
women = pd.read_csv('athletes-women.csv')
men = pd.read_csv('athletes-men.csv')
athletes = pd.read_csv('athletes-all.csv')

# athletes who made it into the games
games = pd.concat([women, men])

,athlete_id,name,rank,region,longitude,latitude,age,height,weight,run400,...,fran,candj,snatch,deadlift,backsq,pullups,coach,certification,multiple,howlong
0,119227,DelainaSnider,9,Canada West (British Columbia),-125.9066,49.1530,34,66,152,67,...,53,200,150,345,280,45,1,0,1,3
1,7716,CamilleLeblanc-Bazinet,1,South West (Virginia),-76.2875,36.7682,26,62,130,80,...,124,230,190,300,310,80,1,1,1,3
2,21024,SarahScholl,4,Atlantic,-87.3595,36.5298,31,68,134,58,...,125,195,155,305,295,40,0,1,0,3
3,51985,KatrinTanjaDavidsdottir,1,"Europe (Reykjavik, Iceland)",-21.9426,64.1466,21,67,152,67,...,128,216,187,309,254,70,1,0,1,3
4,385450,TalaynaFortunato,3,South East,-81.7948,26.1420,34,67,150,69,...,128,215,190,315,275,55,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,521299,BrianMontgomery,19,South East (Florida),-81.5158,27.6648,51,69,178,66,...,203,255,165,425,330,55,1,1,1,2
282,47551,MattDettmann,19,North Central (Missouri),-91.8318,37.9643,41,70,179,68,...,339,225,185,345,305,55,0,1,0,0
283,407586,WilliamSanchez,117,Costa Rica,-83.7534,9.7489,47,70,217,72,...,390,315,245,485,420,34,1,0,0,0
284,250862,DavidGantz,20,North Central (Illinois),-89.3985,40.6331,54,67,167,74,...,188,230,170,405,345,55,0,0,0,0


In [35]:
df = athletes.merge(games.drop_duplicates(), on=['athlete_id'], 
                   how='left', indicator=True)

In [43]:
# athletes that did not make it
temp = df.loc[df['_merge'] == 'left_only']
# athletes that made it
temp2 = df.loc[df['_merge'] == 'both']

In [69]:
# taking only the columns we want
# df_all = df[['athlete_id', 'age_y','age_x','fran_x', 'fran_y', 'run5k_x', 'run5k_y','region_y','region_x', 'pullups_x', 'pullups_y', '_merge']]
df_all = df[['athlete_id', 'age_x','fran_x', 'run5k_x', 'region_x', 'pullups_x', '_merge']]

In [70]:
df_all

,athlete_id,age_x,fran_x,run5k_x,region_x,pullups_x,_merge
0,2554.0,24.0,NaN,NaN,South West,NaN,left_only
1,3517.0,42.0,NaN,NaN,NaN,NaN,left_only
2,4691.0,NaN,NaN,NaN,NaN,NaN,left_only
3,5164.0,40.0,211.0,NaN,Southern California,25.0,left_only
4,5286.0,32.0,206.0,1081.0,NaN,50.0,left_only
...,...,...,...,...,...,...,...
423014,574489.0,36.0,NaN,NaN,Latin America,NaN,left_only
423015,585696.0,27.0,NaN,NaN,Australia,NaN,left_only
423016,608828.0,44.0,NaN,NaN,North West,NaN,left_only
423017,628881.0,20.0,NaN,NaN,Europe,NaN,left_only


In [73]:
# res is athletes who have the stats filled out
res = df_all.dropna()


In [74]:
# ones who made it who have stats filled out, 
# TODO: need to figure out how to put the stats that we manually found in this df
res.loc[res['_merge'] == 'both']

,athlete_id,age_x,fran_x,run5k_x,region_x,pullups_x,_merge
145,36133.0,31.0,245.0,1160.0,Mid Atlantic,35.0,both
2919,13006.0,36.0,183.0,1290.0,Mid Atlantic,40.0,both
9912,9321.0,31.0,173.0,1500.0,Australia,22.0,both
12352,39847.0,30.0,4719.0,5395.0,South East,2.0,both
15609,2901.0,37.0,140.0,1260.0,Europe,44.0,both
...,...,...,...,...,...,...,...
162569,67688.0,25.0,330.0,1320.0,South West,25.0,both
162663,1805.0,38.0,226.0,1230.0,South Central,40.0,both
165702,116860.0,23.0,240.0,1400.0,North West,15.0,both
166125,238819.0,31.0,298.0,1559.0,North East,21.0,both
